In [1]:
import pandas as pd
users = pd.read_csv('users.csv')
purchases = pd.read_csv('purchases.csv')

In [6]:
print(users.head(),  "\n\n\n",purchases.head())

        uid              reg_date device gender country  age
0  54030035  2017-06-29T00:00:00Z    and      M     USA   19
1  72574201  2018-03-05T00:00:00Z    iOS      F     TUR   22
2  64187558  2016-02-07T00:00:00Z    iOS      M     USA   16
3  92513925  2017-05-25T00:00:00Z    and      M     BRA   41
4  99231338  2017-03-26T00:00:00Z    iOS      M     FRA   59 


          date       uid  price
0  2017-07-10  41195147    499
1  2017-07-15  41195147    499
2  2017-11-12  41195147    599
3  2017-09-26  91591874    299
4  2017-12-01  91591874    599


In [8]:
df = purchases.merge(users, how = "inner", on = "uid")

tot_price = df.groupby(["country", "device", "gender", "age"]).agg({"price" : sum})
tot_price.head(200)

price
country device gender age       
BRA     and    F      15   33824
                      16   31619
                      17   20352
                      18   20047
                      19   21352
                      20   22640
                      21   20951
                      22   21346
                      23   15059
                      24    7982
                      25   15264
                      26   16260
                      27    5187
                      28    7183
                      29    8879

In [9]:
# sorted tot_price df

agg_df = tot_price.sort_values(by = "price", ascending = False)
agg_df.head()

price
country device gender age       
USA     and    M      15   61550
BRA     and    M      19   45392
DEU     iOS    F      16   41602
USA     and    F      17   40004
               M      23   39802

In [10]:
# change indexes' names
agg_df = agg_df.reset_index()
agg_df.head()

,country,device,gender,age,price
0,USA,and,M,15,61550
1,BRA,and,M,19,45392
2,DEU,iOS,F,16,41602
3,USA,and,F,17,40004
4,USA,and,M,23,39802


In [11]:
# adding age_cat
agg_df["age_cat"] = pd.cut(agg_df["age"], bins=[0, 18, 23, 30, 40, 75], labels=['0_18', '19_23', '24_30', '31_40', '41_75'])
agg_df.head()

,country,device,gender,age,price,age_cat
0,USA,and,M,15,61550,0_18
1,BRA,and,M,19,45392,19_23
2,DEU,iOS,F,16,41602,0_18
3,USA,and,F,17,40004,0_18
4,USA,and,M,23,39802,19_23


In [12]:
# customer_level_based
agg_df["customer_level_based"] = [row[0] + "_" + row[1].upper() + "_" + row[2] + "_" + row[5] for row in agg_df.values]
agg_df.head()

,country,device,gender,age,price,age_cat,customer_level_based
0,USA,and,M,15,61550,0_18,USA_AND_M_0_18
1,BRA,and,M,19,45392,19_23,BRA_AND_M_19_23
2,DEU,iOS,F,16,41602,0_18,DEU_IOS_F_0_18
3,USA,and,F,17,40004,0_18,USA_AND_F_0_18
4,USA,and,M,23,39802,19_23,USA_AND_M_19_23


In [13]:
agg_df = agg_df[["customer_level_based","price"]]
agg_df.head()

,customer_level_based,price
0,USA_AND_M_0_18,61550
1,BRA_AND_M_19_23,45392
2,DEU_IOS_F_0_18,41602
3,USA_AND_F_0_18,40004
4,USA_AND_M_19_23,39802


In [14]:
agg_df = agg_df.groupby("customer_level_based").agg({"price":"mean"})
#agg_df = agg_df.groupby("customer_level_based").agg({"price":["mean","max","min"]})
agg_df.reset_index(inplace = True)

In [15]:
agg_df["segment"] = pd.qcut(agg_df["price"], 4 ,labels = ["D","C", "B", "A"])
agg_df.groupby("segment").agg({"price" : "mean"}).reset_index(inplace = True)
agg_df.head()

,customer_level_based,price,segment
0,BRA_AND_F_0_18,26460.50,A
1,BRA_AND_F_19_23,20269.60,A
2,BRA_AND_F_24_30,9363.00,A
3,BRA_AND_F_31_40,8354.75,B
4,BRA_AND_F_41_75,4170.60,C


In [19]:
# A female user between the ages of 31 and 40, whose country is in Germany, has an iOS platform.
new_user = "DEU_IOS_F_31_40"

In [20]:
new_customer = agg_df[agg_df["customer_level_based"] == new_user]
new_customer[segment]

,customer_level_based,price,segment
50,DEU_IOS_F_31_40,2992.75,D


In [22]:
new_customer["segment"]

50    D
Name: segment, dtype: category
Categories (4, object): ['D' < 'C' < 'B' < 'A']